In [78]:
import numpy as np
import pandas as pd
import math

In [79]:
def get_m2_max_abs_second_derivative(func_d2, a, b, num_samples=1000):
    """
    Estimates M2 = max|f''(x)| on [a, b] by sampling.
    """
    x_test = np.linspace(a, b, num_samples)
    y_test = np.abs(func_d2(x_test))
    return np.max(y_test)

# Trapezoid-Join in Approximate Definite Integral

## Algorithm

* Input
1.  **Function**: $f(x)$ defined and continuous on $[a, b]$.
2.  **Interval**: $[a, b]$.
3.  **Error Tolerance**: $\epsilon$ (maximum allowable error).

* Steps

**1. Determine Number of Sub-intervals ($n$)**
   * Calculate the maximum absolute value of the second derivative on the interval:
       $$M_2 = \max_{x \in [a,b]} |f''(x)|$$
   * Using the error bound formula $|E| \le \frac{M_2 (b-a)^3}{12 n^2}$, solve for the minimum integer $n$ such that the error is less than $\epsilon$:
       $$n > \sqrt{\frac{M_2 (b-a)^3}{12 \epsilon}}$$
   * Set $n = \lceil \text{calculated value} \rceil$ (round up to the nearest integer).

**2. Define Step Size ($h$)**
   * Calculate the width of each sub-interval:
       $$h = \frac{b - a}{n}$$

**3. Discretization**
   * Generate $n+1$ points ($x_k$) and their corresponding function values ($y_k$):
       For $k = 0, 1, \dots, n$:
       $$x_k = a + k \cdot h$$
       $$y_k = f(x_k)$$

**4. Compute Integral Approximation**
   * Apply the Composite Trapezoidal formula:
       $$I \approx \frac{h}{2} \left[ y_0 + y_n + 2 \sum_{k=1}^{n-1} y_k \right]$$

## Output
* **Approximate Integral**: $I \approx \int_a^b f(x) \, dx$

In [80]:
def loopTimes (f, d2f, a, b, eps):
    # Step 1: Calculate M2
    M2 = get_m2_max_abs_second_derivative(d2f, a, b)

    # Step 2: Determine number of segments n
    # Error <= (M2 * (b-a)^3) / (12 * n^2) < eps
    # n > sqrt( (M2 * (b-a)^3) / (12 * eps) )
    numerator = M2 * (b - a)**3
    denominator = 12 * eps
    n_float = np.sqrt(numerator / denominator)
    n = math.ceil(n_float)

    return numerator, denominator, n_float, n

In [81]:
def solve_trapezoidal(f, d2f, a, b, eps):
    M2 = get_m2_max_abs_second_derivative(d2f, a, b)
    numerator, denominator, n_float, n = loopTimes(f, d2f, a, b, eps)
    h = (b - a) / n

    # Step 4: Generate points x_k and values y_k
    # x points
    x_values = np.linspace(a, b, n + 1)
    # y points
    y_values = f(x_values)

    # Display Table using Pandas
    data = {
        'k': range(n + 1),
        'x_k': x_values,
        'y_k': y_values
    }
    df = pd.DataFrame(data)

    # Step 5: Apply Trapezoidal Formula
    # I = h/2 * [y0 + yn + 2 * sum(y_1 to y_{n-1})]
    sum_middle = np.sum(y_values[1:-1])
    y_start = y_values[0]
    y_end = y_values[-1]
    
    I = (h / 2) * (y_start + y_end + 2 * sum_middle)
    
    return df, I

## Result

In [82]:
f = lambda x: 1/(x**2+1)

d2f = lambda x: (2*(3*x**2-1))/(x**6+3*x**4+3*x**2+1)

a = 0.0       # Start of interval
b = 2.0       # End of interval
eps = 1e-6 # Desired maximum error

print(f"--- Trapezoidal Rule Calculation ---")
print(f"Goal: Integrate f(x) on [{a}, {b}] with error < {eps}\n")

--- Trapezoidal Rule Calculation ---
Goal: Integrate f(x) on [0.0, 2.0] with error < 1e-06



In [83]:
M2 = get_m2_max_abs_second_derivative(d2f, a, b)
print(f"Step 1: Estimate max|f''(x)| (M2)")
print(f"M2 ≈ {M2:.6f}")

Step 1: Estimate max|f''(x)| (M2)
M2 ≈ 2.000000


In [84]:
numerator, denominator, n_float, n = loopTimes(f, d2f, a, b, eps)
print(f"\nStep 2: Calculate minimum n")
print(f"n > sqrt({numerator:.6f} / {denominator:.6f}) = {n_float:.6f}")
print(f"Chosen n = {n}")


Step 2: Calculate minimum n
n > sqrt(16.000000 / 0.000012) = 1154.700538
Chosen n = 1155


In [85]:
h = (b - a) / n
print(f"\nStep 3: Calculate step size h")
print(f"h = ({b} - {a}) / {n} = {h:.6f}")


Step 3: Calculate step size h
h = (2.0 - 0.0) / 1155 = 0.001732


In [86]:
df, I = solve_trapezoidal(f, d2f, a, b, eps)
print(f"\nStep 4: Table of values (x_k, y_k)")
df



Step 4: Table of values (x_k, y_k)


,k,x_k,y_k
0,0,0.000000,1.000000
1,1,0.001732,0.999997
2,2,0.003463,0.999988
3,3,0.005195,0.999973
4,4,0.006926,0.999952
...,...,...,...
1151,1151,1.993074,0.201112
1152,1152,1.994805,0.200834
1153,1153,1.996537,0.200555
1154,1154,1.998268,0.200277


In [87]:
print(f"\nStep 5: Final Calculation")
print(f"Approximate Integral I = {I:.8f}")


Step 5: Final Calculation
Approximate Integral I = 1.10714868
